In [43]:
# some useful mysklearn package import statements and reloads
import importlib

import pandas as pd
import numpy as np
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation


In [44]:
import pandas as pd
from sklearn.model_selection import KFold

# Load data
data = pd.read_csv('normalized_flights.csv')

# Compute flight delay (in minutes, assuming sched_dep_time and dep_time are timestamps or integers)
data['flight_delay'] = data['sched_dep_time'] - data['dep_time']

# Extract meaningful features for predicting delays
X = data[[
    'year', 'month', 'day', 'hour', 'minute',  # Time-based features
    'carrier', 'flight', 'tailnum',            # Operational features
    'origin', 'dest', 'distance', 'air_time', # Location & flight characteristics
    'dep_delay', 'sched_dep_time'             # Existing delay and timing
]]

# Target variable: flight delay
y = data['flight_delay']

# Initialize K-Fold cross-validation (10 splits)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for performance metrics
accuracies, error_rates, precisions, recalls, f1_scores, confusion_matrices = [], [], [], [], [], []


In [52]:
import numpy as np
import pandas as pd

# Load the dataset (assuming `data` is a pandas DataFrame containing the provided sample data)
data = pd.read_csv('normalized_flights.csv') 

# Define delay categories
def categorize_delay(delay):
    if delay <= 0:
        return 0
    elif 0 < delay <= 30:
        return 1
    elif 30 < delay <= 60:
        return 2
    elif 60 < delay <= 120:
        return 3
    elif 120 < delay <= 180:
        return 4
    else:
        return 5

# Map delays to categories
data['delay_category'] = data['arr_delay'].apply(categorize_delay)

# Prepare features (X) and labels (y)
X = data[['dep_time', 'sched_dep_time', 'dep_delay', 'air_time', 'distance']].fillna(0).values
y = data['delay_category'].values

In [53]:
X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10 (assuming myevaluation provides this utility)
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics (accuracy, precision, recall, f1, confusion matrices, etc.)
knn_accuracies, knn_precisions, knn_recalls, knn_f1s, knn_conf_matrices = [], [], [], [], []
dummy_accuracies, dummy_precisions, dummy_recalls, dummy_f1s, dummy_conf_matrices = [], [], [], [], []

In [ ]:
# Initialize lists to gather true and predicted values across all folds
all_y_true = []
all_y_pred = []

# Loop over each fold
for train_index, test_index in kf:
    knn_X_train, knn_X_test = X[train_index], X[test_index]
    knn_y_train, knn_y_test = y[train_index], y[test_index]

    # Initialize and fit MyKNeighborsClassifier
    knn_model = MyKNeighborsClassifier()
    knn_model.fit(knn_X_train.tolist(), knn_y_train.tolist())
    
    # Make predictions
    knn_y_pred = knn_model.predict(knn_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(knn_y_test, knn_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(knn_y_test, knn_y_pred, pos_label="yes")
    recall_score = myevaluation.binary_recall_score(knn_y_test, knn_y_pred, pos_label="yes")
    f1 = myevaluation.binary_f1_score(knn_y_test, knn_y_pred, pos_label="yes")
    confusion_matrix = myevaluation.confusion_matrix(knn_y_test, knn_y_pred, labels=["yes", "no"])

    # Store fold metrics
    knn_accuracies.append(accuracy)
    knn_precisions.append(precision_score)
    knn_recalls.append(recall_score)
    knn_f1s.append(f1)
    knn_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(knn_y_test)
    all_y_pred.extend(knn_y_pred)

# Calculate the final confusion matrix using all folds' predictions
labels = [0, 1, 2, 3, 4, 5]
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

In [ ]:
### kNN Performance Evaluation & Confusion Matrix

In [ ]:
print("===========================================")
print("kNN Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(knn_accuracies):.2f}")
print(f"Error: {error:.2f}")
print(f"Precision: {np.mean(knn_precisions):.2f}")
print(f"Recall: {np.mean(knn_recalls):.2f}")
print(f"F1 Score: {np.mean(knn_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

kNN Classifier Performance
Accuracy: nan


/opt/conda/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


NameError: name 'error' is not defined

In [49]:
# Convert X and y to NumPy arrays
X = np.array(X)
y = np.array(y)

# Initialize StratifiedKFold with k=10
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics
dummy_accuracies, dummy_precisions, dummy_recalls, dummy_f1s, dummy_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Loop over each fold in Stratified K-Fold
for train_index, test_index in kf:
    dummy_X_train, dummy_X_test = X[train_index], X[test_index]
    dummy_y_train, dummy_y_test = y[train_index], y[test_index]

    # Initialize and fit MyDummyClassifier
    dummy_model = MyDummyClassifier(strategy="most_frequent")
    dummy_model.fit(dummy_y_train.tolist())  # Fit the model on the training labels only
    
    # Make predictions on the test set
    dummy_y_pred = dummy_model.predict(dummy_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(dummy_y_test, dummy_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(dummy_y_test, dummy_y_pred, pos_label="yes")
    recall_score = myevaluation.binary_recall_score(dummy_y_test, dummy_y_pred, pos_label="yes")
    f1 = myevaluation.binary_f1_score(dummy_y_test, dummy_y_pred, pos_label="yes")
    confusion_matrix = myevaluation.confusion_matrix(dummy_y_test, dummy_y_pred, labels=["yes", "no"])

    # Store metrics for this fold
    dummy_accuracies.append(accuracy)
    dummy_precisions.append(precision_score)
    dummy_recalls.append(recall_score)
    dummy_f1s.append(f1)
    dummy_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(dummy_y_test)
    all_y_pred.extend(dummy_y_pred)

# Calculate the final confusion matrix using all folds' predictions
labels = ["yes", "no"]
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

TypeError: MyDummyClassifier.__init__() got an unexpected keyword argument 'strategy'

### Dummy Performance Evaluation & Confusion Matrix

In [ ]:
print("===========================================")
print("Dummy Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(dummy_accuracies):.2f}")
print(f"Error: {error:.2f}")
print(f"Precision: {np.mean(dummy_precisions):.2f}")
print(f"Recall: {np.mean(dummy_recalls):.2f}")
print(f"F1 Score: {np.mean(dummy_f1s):.2f}\n")


# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

In [51]:
# Convert X and y to NumPy arrays
X = np.array(X)
y = np.array(y)

# Initialize StratifiedKFold with k=10
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics
nb_accuracies, nb_precisions, nb_recalls, nb_f1s, nb_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Loop over each fold in Stratified K-Fold
for train_index, test_index in kf:
    nb_X_train, nb_X_test = X[train_index], X[test_index]
    nb_y_train, nb_y_test = y[train_index], y[test_index]

    # Initialize and fit MyNaiveBayesClassifier
    nb_model = MyNaiveBayesClassifier()
    nb_model.fit(nb_X_train.tolist(), nb_y_train.tolist())
    
    # Make predictions on the test set
    nb_y_pred = nb_model.predict(nb_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(nb_y_test, nb_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(nb_y_test, nb_y_pred, pos_label="yes")
    recall_score = myevaluation.binary_recall_score(nb_y_test, nb_y_pred, pos_label="yes")
    f1 = myevaluation.binary_f1_score(nb_y_test, nb_y_pred, pos_label="yes")
    confusion_matrix = myevaluation.confusion_matrix(nb_y_test, nb_y_pred, labels=["yes", "no"])

    # Store metrics for this fold
    nb_accuracies.append(accuracy)
    nb_precisions.append(precision_score)
    nb_recalls.append(recall_score)
    nb_f1s.append(f1)
    nb_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(nb_y_test)
    all_y_pred.extend(nb_y_pred)

# Calculate the final confusion matrix using all folds' predictions
labels = [0, 1, 2, 3, 4, 5]
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

KeyError: 0